# Predicting How Many Days a Customer Will Rent a DVD Using Machine Learning — Regression

<img src="https://www.pressenterprise.com/wp-content/uploads/migration/nrf/nrfumn-b88430219z.120150713111123000gt4aq630.10.jpg?w=1200" alt="drawing" width="1400"/>

This project aims to figure out how many days a customer will rent a DVD for based on some features. We are going to try out some regression models which will help predict the number of days a customer will rent a DVD using machine learning techniques. Specifically, we are using **regression** machine learning models.

## Data

The data is provided in the CSV file `rental_info.csv`. It has the following features:

- `rental_date`: The date (and time) the customer rents the DVD.
- `return_date`: The date (and time) the customer returns the DVD.
- `amount`: The amount paid by the customer for renting the DVD.
- `amount_2`: The square of `amount`.
- `rental_rate`: The rate at which the DVD is rented.
- `rental_rate_2`: The square of `rental_rate`.
- `release_year`: The year the movie being rented was released.
- `length`: Length of the movie being rented, in minutes.
- `length_2`: The square of `length`.
- `replacement_cost`: The amount it would cost the company to replace the DVD.
- `special_features`: Any special features, such as trailers or deleted scenes.
- `NC-17`, `PG`, `PG-13`, `R`: Dummy variables indicating the movie's rating. Each is 1 if the movie has that rating, 0 otherwise. (One category is dropped as the reference.)
